# Subsystem Sensitivity Analysis


## Load Model


In [ ]:
import pysd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import numpy as np

current_dir = Path.cwd()
BASE_DIR = current_dir.parent if current_dir.name == 'notebooks' else current_dir
os.chdir(BASE_DIR)

# Load the default parameters
with open(BASE_DIR / 'params.yaml', 'r') as file:
    BASE_PARAMS = yaml.safe_load(file)

model = pysd.load('src/model.py')


## Calculate Tax Frontier


In [ ]:
from src.utils.max_tax import calculate_max_viable_tax

tax_frontier = calculate_max_viable_tax(model, BASE_PARAMS)

print(tax_frontier)


## Sensitivity & Leverage Points


In [ ]:
from src.utils.sensitivity import one_at_a_time_sensitivity_analysis

# Tax levels: we choose levels within the range of the tax frontier
tax_levels_all = np.linspace(BASE_PARAMS["carbon_tax_rate"], tax_frontier, 5)
# Remove specific tax levels: 1447 and 3763
tax_levels = tax_levels_all[~np.isclose(tax_levels_all, 1447, atol=10) & ~np.isclose(tax_levels_all, 3763, atol=10)]

key_params_policy = {

    # 1. Fuel carbon intensity (fuel supply decarbonisation)
    # e.g. refinery subsidies, LCFS, synthetic fuels
    "Fuel carbon intensity": {
        "baseline_ci": np.linspace(0.00268 * 0.85, 0.00268 * 1.05, 5),
        "carbon_content_of_fuel": np.linspace(0.00268 * 0.90, 0.00268 * 1.00, 5),
        "max_reduction_ci": np.linspace(0.30, 0.60, 5),
        "tax_scale": np.linspace(2000, 5000, 5),
    },

    # 2. Fuel efficiency (vehicles & operations)
    # e.g. vehicle upgrades, logistics optimisation
    "Fuel efficiency": {
        "baseline_fuel_efficiency": np.linspace(2.84 * 0.85, 2.84 * 1.10, 5),
        "max_efficiency": np.linspace(1.15, 1.40, 5),
        "cost_pressure_sensitivity": np.linspace(0.3, 1.0, 5),
        "cost_pressure_at_max_improvement": np.linspace(0.7, 1.3, 5),
    },

    # 3. Non-fuel operating costs (fixed & quasi-fixed costs)
    # e.g. wage support, toll relief, vehicle tax relief
    "Non-fuel operating costs": {
        "nonfuel_cost_per_km": np.linspace(90 * 0.70, 90 * 1.05, 5),
    },

    # 4. Fuel market price exposure
    # e.g. fuel subsidies, price stabilisation
    "Fuel market price": {
        "pretax_fuel_price": np.linspace(108 * 0.80, 108 * 1.20, 5),
    },

    # 5. Price pass-through / market structure
    # e.g. contract regulation, competition policy
    "Price pass-through": {
        "desired_passthrough_share": np.linspace(0.30, 0.80, 5),
    },

    # 6. Demand responsiveness
    # e.g. modal shift, supply-chain restructuring
    "Demand responsiveness": {
        "baseline_demand": np.linspace(19e9 * 0.9, 19e9 * 1.10, 5),
        "elasticity_sr": np.linspace(-0.30, -0.10, 5),
        "elasticity_lr": np.linspace(-1.00, -0.40, 5),
    },
}

sensitivity_params = BASE_PARAMS.copy()

# Create mapping from parameter names to subsystem names
param_to_subsystem = {}
for subsystem_name, params_dict in key_params_policy.items():
    for param_name in params_dict.keys():
        param_to_subsystem[param_name] = subsystem_name

# Flatten the nested structure for the sensitivity analysis function
key_params_flat = {}
for subsystem_name, params_dict in key_params_policy.items():
    for param_name, param_values in params_dict.items():
        key_params_flat[param_name] = param_values

model.run(params=sensitivity_params)

results = one_at_a_time_sensitivity_analysis(model, sensitivity_params, key_params_flat, tax_levels)


## 2D Influence Plot: Emissions vs Profitability


In [ ]:
# Calculate influence on both emissions and profitability for each subsystem at each tax level
# Aggregate influences across all parameters that belong to the same subsystem
influence_data = []
for tax in tax_levels:
    tax_data = results[results['tax'] == tax]
    
    # Group by subsystem
    for subsystem_name in key_params_policy.keys():
        # Get all parameters belonging to this subsystem
        subsystem_params = key_params_policy[subsystem_name].keys()
        
        # Aggregate influence across all parameters in this subsystem
        subsystem_profit_ranges = []
        subsystem_co2_ranges = []
        
        for param_name in subsystem_params:
            param_data = tax_data[tax_data['param_name'] == param_name]
            
            if len(param_data) > 0:
                param_data_profit = param_data['profit_change_pct']
                param_data_co2 = param_data['co2_reduction_pct']
                
                # Calculate range for this parameter
                profit_range = abs(param_data_profit.max() - param_data_profit.min())
                co2_range = abs(param_data_co2.max() - param_data_co2.min())
                
                subsystem_profit_ranges.append(profit_range)
                subsystem_co2_ranges.append(co2_range)
        
        # Aggregate: use maximum range across all parameters in the subsystem
        # (alternative: could use sum or mean)
        if len(subsystem_profit_ranges) > 0:
            influence_profit = max(subsystem_profit_ranges)  # Maximum influence across parameters
            influence_co2 = max(subsystem_co2_ranges)  # Maximum influence across parameters
            
            influence_data.append({
                'tax': tax,
                'param_name': subsystem_name,  # Now this is actually the subsystem name
                'influence_profit': influence_profit,
                'influence_co2': influence_co2
            })

# Create DataFrame
influence_df = pd.DataFrame(influence_data)

# Normalize within each tax level so that sum of all influences = 1 for each axis
# For each tax level τ: Σᵢ I^emissions_{i,τ} = 1 and Σᵢ I^profit_{i,τ} = 1
influence_df['influence_profit_plot'] = 0.0
influence_df['influence_co2_plot'] = 0.0

for tax in tax_levels:
    tax_mask = influence_df['tax'] == tax
    tax_data = influence_df[tax_mask]
    
    if len(tax_data) > 0:
        # Sum of all influences at this tax level
        profit_sum = tax_data['influence_profit'].sum()
        co2_sum = tax_data['influence_co2'].sum()
        
        # Normalize so that sum = 1 for each axis
        if profit_sum > 0:
            influence_df.loc[tax_mask, 'influence_profit_plot'] = (
                tax_data['influence_profit'] / profit_sum
            ).values
        else:
            # If all zero, assign equal shares
            n_subsystems = len(tax_data)
            influence_df.loc[tax_mask, 'influence_profit_plot'] = 1.0 / n_subsystems
        
        if co2_sum > 0:
            influence_df.loc[tax_mask, 'influence_co2_plot'] = (
                tax_data['influence_co2'] / co2_sum
            ).values
        else:
            # If all zero, assign equal shares
            n_subsystems = len(tax_data)
            influence_df.loc[tax_mask, 'influence_co2_plot'] = 1.0 / n_subsystems

# Get unique subsystems and tax levels for colors and shapes
subsystems = influence_df['param_name'].unique()
tax_levels_unique = sorted(influence_df['tax'].unique())

# Create color map for subsystems
colors = plt.cm.tab20(np.linspace(0, 1, len(subsystems)))
subsystem_colors = dict(zip(subsystems, colors))

# Create shape map for tax levels
shapes = ['o', 's', '^', 'D', 'v', 'p', '*', 'h', 'H', '8']
tax_shapes = dict(zip(tax_levels_unique, shapes[:len(tax_levels_unique)]))

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))

# Plot each point
for idx, row in influence_df.iterrows():
    subsystem = row['param_name']
    tax = row['tax']
    
    ax.scatter(
        row['influence_co2_plot'],
        row['influence_profit_plot'],
        c=[subsystem_colors[subsystem]],
        marker=tax_shapes[tax],
        s=150,
        alpha=0.7,
        edgecolors='black',
        linewidths=1.5
    )

# Add labels and title
ax.set_xlabel('Relative Influence on Emissions\n(Σᵢ I^emissions_{i,τ} = 1 for each tax level τ)', fontsize=12)
ax.set_ylabel('Relative Influence on Profitability\n(Σᵢ I^profit_{i,τ} = 1 for each tax level τ)', fontsize=12)
ax.set_title('Subsystem Sensitivity: Emissions vs Profitability\n(Shapes = Tax Levels, Colors = Subsystems)', fontsize=14, fontweight='bold')

# Add grid
ax.grid(True, alpha=0.3, linestyle='--')

# Create legend for subsystems (colors)
from matplotlib.patches import Patch
legend_elements_subsystems = [Patch(facecolor=subsystem_colors[sys], label=sys) for sys in subsystems]
legend1 = ax.legend(handles=legend_elements_subsystems, title='Subsystems', loc='upper left', bbox_to_anchor=(1.02, 1))

# Create legend for tax levels (shapes)
from matplotlib.lines import Line2D
legend_elements_tax = [Line2D([0], [0], marker=tax_shapes[tax], color='black', 
                              label=f'Tax: {tax:.0f} ¥/tCO₂', markersize=10, linestyle='None') 
                       for tax in tax_levels_unique]
legend2 = ax.legend(handles=legend_elements_tax, title='Tax Levels', loc='lower left', bbox_to_anchor=(1.02, 0))
ax.add_artist(legend1)  # Re-add the first legend

plt.tight_layout()
plt.savefig(f"{BASE_DIR}/figures/results/subsystem_sensitivity_2d.png", dpi=300, bbox_inches='tight')
plt.show()


## Subplots by Tax Level


In [ ]:
# Create subplots for each tax level separately
fig, axes = plt.subplots(1, len(tax_levels_unique), figsize=(6 * len(tax_levels_unique), 6))

# If only one tax level, make axes iterable
if len(tax_levels_unique) == 1:
    axes = [axes]

for idx, tax in enumerate(tax_levels_unique):
    ax = axes[idx]
    tax_data = influence_df[influence_df['tax'] == tax]
    
    # Plot each subsystem for this tax level
    for subsystem in subsystems:
        subsystem_data = tax_data[tax_data['param_name'] == subsystem]
        
        if len(subsystem_data) > 0:
            row = subsystem_data.iloc[0]
            ax.scatter(
                row['influence_co2_plot'],
                row['influence_profit_plot'],
                c=[subsystem_colors[subsystem]],
                marker='o',
                s=200,
                alpha=0.7,
                edgecolors='black',
                linewidths=2,
                label=subsystem
            )
    
    # Add labels and title for each subplot
    ax.set_xlabel('Relative Influence on Emissions', fontsize=11)
    if idx == 0:  # Only add y-label to leftmost subplot
        ax.set_ylabel('Relative Influence on Profitability', fontsize=11)
    ax.set_title(f'Tax Level: {tax:.0f} ¥/tCO₂', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xlim([0, 0.7])
    ax.set_ylim([0, 0.7])
    
    # Add diagonal reference line
    ax.plot([0, 0.7], [0, 0.7], 'k--', alpha=0.2, linewidth=1, label='Equal influence')

# Add overall title
fig.suptitle('Subsystem Sensitivity by Tax Level\n(Σᵢ I^emissions_{i,τ} = 1 and Σᵢ I^profit_{i,τ} = 1 for each tax level τ)', 
             fontsize=14, fontweight='bold', y=1.02)

# Add legend for subsystems (only on the rightmost subplot)
legend_elements_subsystems = [Patch(facecolor=subsystem_colors[sys], label=sys) for sys in subsystems]
axes[-1].legend(handles=legend_elements_subsystems, title='Subsystems', 
                loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=9)

plt.tight_layout()
plt.savefig(f"{BASE_DIR}/figures/results/subsystem_sensitivity_by_tax_level.png", dpi=300, bbox_inches='tight')
plt.show()
